In [2]:
from sklearn.datasets import load_svmlight_file
import numpy as np
import math
import random
import matplotlib.pyplot as plt

In [3]:
X_train, y_train = load_svmlight_file("a9a", n_features=123)
X_train = X_train.toarray()
X_val, y_val = load_svmlight_file("a9a.t", n_features=123)
X_val = X_val.toarray()

In [4]:
n_samples_train, n_features = X_train.shape
X_train = np.column_stack((X_train, np.ones((n_samples_train, 1))))
y_train = y_train.reshape((-1, 1))
for i in range(y_train.shape[0]):
    if y_train[i, 0] == -1:
        y_train[i, 0] = 0

In [5]:
n_samples_val, n_features = X_val.shape
X_val = np.column_stack((X_val, np.ones((n_samples_val, 1))))
y_val = y_val.reshape((-1, 1))
for i in range(y_val.shape[0]):
    if y_val[i, 0] == -1:
        y_val[i, 0] = 0


In [6]:
max_epoch = 15
batch_size = 200
learning_rate = 0.001
C = 0.01

losses_train = []
losses_val = []

w = np.random.random((n_features + 1, 1))#/15


In [7]:
def sigmoid(x, batch_size):
    result = np.mat(np.linspace(0, 0, batch_size)).T
    for i in range(batch_size):
        result[i, 0] = 1 / (1 + math.exp(-x[i, 0]))
    return result

In [8]:
for epoch in range(max_epoch):
    start = random.randrange(0, n_samples_train - batch_size, 1)
    end = start + batch_size
    #loss_train = np.average(sigmoid(np.dot(X_train[start:end, :], w), batch_size) - y_train[start:end, :])
    #print(sigmoid(np.dot(X_train[start:end, :], w), batch_size), y_train[start:end, :])
    #loss_train = np.average(np.log(1 + math.exp(-np.dot(y_train[start:end, :].T, np.dot(X_train[start:end, :], w))))) + C/2 * np.average(np.dot(w.T, w))
    loss_train = np.average(-np.dot(y_train[start:end, :].T, np.log(1/(1+np.exp(-np.dot(X_train[start:end, :], w)))))-np.dot((1-y_train[start:end, :]).T, np.log(1-1/(1+np.exp(-np.dot(X_train[start:end, :], w)))))) / batch_size
    losses_train.append(loss_train)
    #loss_val = np.average(sigmoid(np.dot(X_val, w), n_samples_val) - y_val)
    loss_val = np.average(-np.dot(y_val.T, np.log(1/(1+np.exp(-np.dot(X_val, w)))))-np.dot((1-y_val).T, np.log(1-1/(1+np.exp(-np.dot(X_val, w)))))) / n_samples_val
    losses_val.append(loss_val)
    descent = np.zeros((n_features+1, 1))
    for i in range(batch_size):
        descent += learning_rate * ((sigmoid(np.dot(X_train[start:end, :], w), batch_size) - y_train[start:end, :]).T * X_train[start:end, :]).T
    descent /= batch_size
    w -= descent


In [9]:
plt.figure(figsize=(18, 7))
plt.plot(losses_train, color="r", label="train")
plt.plot(losses_val, color="b", label="validation")
plt.legend()
plt.xlabel("epoch")
plt.ylabel("loss")
plt.title("The graph of loss value varing with the number of iterations")
plt.show()